In [1]:
# Copyright 2018 Anar Amirli
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train/Test Set Construction
In this document, we construct training and test sets from already computed feature sets. The sets are computed according to the layout:

data<br>
├── train_features<br>
├── test_features<br>

In [1]:
from os import listdir
from os.path import join
import pickle
import numpy as np
import pandas as pd
import multiprocessing
from collections import Counter
from random import shuffle
import json
import re

# our common utility functions that are used in different notebooks
from utils import *

pd.set_option('compute.use_bottleneck', True)
pd.set_option('compute.use_numexpr', True)

In [2]:
def describe_dataset(dataset):
    print('Shape: {}\n'.format(dataset.shape))
    
    _, col = dataset.shape
        
    heading = 'Pitch index counts'
    print(heading + '\n' + '-'*len(heading))
    for key, val in sorted(Counter(dataset.values[:, col-3]).items()):
        print('{}\t: {}'.format(int(key), val))

## Pitch IDS

In [3]:
if __name__ == '__main__':

    match_id = 60565
    try:
        features_df = pd.read_csv('../data/general/feature-set/match{}_features_2.csv'.format(match_id))
    except FileNotFoundError:
        pass

    ball_idxs = np.sort(features_df['pitch_index'].unique())
    print('Ball  index: {}'.format(ball_idxs))

    print('Size: {}'.format(len(ball_idxs)))

Ball  index: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
Size: 10


Null index(0) represents all the event categories, apart from the ones we are interested in, that occur when the game stops.

## Dataset Construction

In this section, we construct a combined shuffled dataset for train and set sets, coming from feature data.


### Parameters

When obtaining feature data for a given index, set we should reduce the number of frames of the null ball events in our data set. But cosidering the fact that they are most common, we should keep their numbers close to the number of other most occurred ball location indexes.

* $-$: No-labaled positions intervals would lead to greater label noise.

### Construction
Here we construct the combined dataset from all match data we have in the given data directories.

In [11]:
    # file based constants
    features_dir     = '../data/general/feature-set/'
    features_regex   = re.compile(r'match\d+_features_2.csv')

We speed up the computation by using all the CPU cores via multiprocessing module.

In [12]:
    pool = multiprocessing.Pool()
    df = pd.DataFrame()

    
    features_csv_files = [f for f in listdir(features_dir) if features_regex.match(f)]
    shuffle(features_csv_files)
    print('All matches: {}'.format(len(features_csv_files)))
    features_train_csv = features_csv_files[0:248]
    features_test_csv  = features_csv_files[248:len(features_csv_files)]
    print('Train matches: {}'.format(len(features_train_csv)))
    print('Test matches: {}'.format(len(features_test_csv)))
    
    
    df_train = pd.concat(pool.map(construct_data_set, features_train_csv))
    df_test = pd.concat(pool.map(construct_data_set, features_test_csv))


    print('Train-set shape: {}'.format(df_train.shape))
    print('Test-set shape: {}'.format(df_test.shape))

All matches: 298
Train matches: 248
Test matches: 50
Train-set shape: (569363, 230)
Test-set shape: (110019, 230)


In [13]:
    # shuffling data-sets agin
    # df_train = df_train.drop('homeGoalKeeperY', axis=1)
    
    df_train = df_train.sample(frac=1).reset_index(drop=True)
    df_test = df_test.sample(frac=1).reset_index(drop=True)

### Train Set

In [14]:
    describe_dataset(df_train)
    display(df_train.head())

Shape: (569363, 230)

Pitch index counts
------------------
0	: 174376
1	: 16287
2	: 39363
3	: 15413
4	: 48708
5	: 142437
6	: 49261
7	: 17806
8	: 44841
9	: 20871


,# refX,refY,refSpeed,refDirectX,refDirectY,teamsDBSCANx,teamsDBSCANy,homeDBSCANx,homeDBSCANy,homeMaxX,...,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index,xpos,ypos
0,62.343750,31.109375,1.269531,-1.0,-1.0,57.21875,39.75000,55.406250,40.031250,81.25000,...,0.0000,0.00000,43.96875,36.9375,0.0,0.0,0.0,5.0,67.00000,28.734375
1,15.656250,26.234375,0.199951,1.0,1.0,14.96875,32.59375,14.882812,31.796875,17.31250,...,14.2500,29.31250,0.00000,0.0000,0.0,0.0,0.0,0.0,0.00000,0.000000
2,19.046875,46.593750,4.839844,-1.0,1.0,35.56250,38.84375,32.562500,40.062500,63.46875,...,0.0000,0.00000,0.00000,0.0000,0.0,0.0,0.0,2.0,9.46875,49.031250
3,58.375000,33.281250,1.440430,1.0,-1.0,56.28125,41.28125,52.312500,36.625000,68.75000,...,0.0000,0.00000,45.31250,32.6250,0.0,0.0,0.0,6.0,54.40625,68.000000
4,85.000000,31.484375,0.640137,1.0,-1.0,87.06250,33.43750,86.500000,31.203125,89.56250,...,84.5625,52.28125,0.00000,0.0000,0.0,0.0,0.0,0.0,0.00000,0.000000


### Test Set

In [15]:
    describe_dataset(df_test)
    display(df_test.head())

Shape: (110019, 230)

Pitch index counts
------------------
0	: 35361
1	: 3510
2	: 7838
3	: 2816
4	: 9098
5	: 26959
6	: 9202
7	: 3350
8	: 7944
9	: 3941


,# refX,refY,refSpeed,refDirectX,refDirectY,teamsDBSCANx,teamsDBSCANy,homeDBSCANx,homeDBSCANy,homeMaxX,...,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index,xpos,ypos
0,64.0000,42.81250,2.210938,-1.0,1.0,65.00000,40.187500,62.406250,40.437500,81.68750,...,48.062500,40.156250,59.03125,63.18750,0.000,0.0000,0.000000,8.0,90.0625,48.59375
1,39.5625,44.53125,0.930176,-1.0,1.0,34.28125,43.968750,34.187500,44.281250,67.93750,...,31.421875,56.375000,0.00000,0.00000,0.000,0.0000,0.000000,6.0,60.5625,63.40625
2,88.7500,23.40625,0.049988,-1.0,-1.0,97.43750,30.062500,97.750000,27.640625,101.81250,...,0.000000,0.000000,0.00000,0.00000,0.000,0.0000,0.000000,0.0,0.0000,0.00000
3,66.2500,22.65625,1.769531,-1.0,1.0,59.37500,32.562500,53.156250,31.609375,74.43750,...,0.000000,0.000000,59.21875,24.21875,49.625,37.9375,3.820312,5.0,44.1875,45.53125
4,31.3125,27.96875,1.839844,1.0,-1.0,18.78125,29.421875,17.046875,28.187500,53.71875,...,28.203125,26.109375,16.62500,45.59375,0.000,0.0000,0.000000,0.0,0.0000,0.00000


## Exporting

In [16]:
    df_train.to_csv('../data/general/{dataset}/all_{dataset}_3.csv'.format(dataset='train'), index=False)
    df_test.to_csv('../data/general/{dataset}/all_{dataset}_3.csv'.format(dataset='test'), index=False)

In [14]:
    df_train[1:10]

,# refX,refY,refSpeed,refDirectX,refDirectY,teamsDBSCANx,teamsDBSCANy,homeDBSCANx,homeDBSCANy,homeMaxX,...,awayCenterForwardAvgY,awayCenterForwardAvgSpeed,awayCenterForwardSlowAvgX,awayCenterForwardSlowAvgY,awayCenterForwardHirAvgX,awayCenterForwardHirAvgY,awayCenterForwardMaxSprintX,awayCenterForwardMaxSprintY,awayCenterForwardMaxSprintSpeed,pitch_index
1,52.187500,21.234375,5.558594,1.0,-1.0,41.406250,23.109375,39.218750,17.343750,71.0625,...,23.093750,5.523438,0.000000,0.000000,0.000000,0.00000,66.3125,10.156250,7.511719,4.0
2,48.218750,34.031250,2.640625,-1.0,-1.0,52.625000,23.109375,50.687500,13.921875,68.0625,...,10.351562,3.839844,0.000000,0.000000,0.000000,0.00000,50.7500,10.351562,3.839844,4.0
3,23.015625,32.406250,1.450195,-1.0,1.0,12.015625,25.515625,11.921875,25.937500,52.3125,...,34.000000,2.259766,0.000000,0.000000,11.226562,34.00000,0.0000,0.000000,0.000000,5.0
4,85.062500,40.812500,1.410156,1.0,1.0,92.437500,44.406250,82.000000,45.000000,100.1250,...,44.437500,0.040009,54.531250,44.437500,0.000000,0.00000,0.0000,0.000000,0.000000,0.0
5,77.562500,23.515625,1.110352,1.0,-1.0,83.375000,32.562500,80.562500,29.828125,87.8750,...,47.125000,1.344727,68.187500,47.125000,0.000000,0.00000,0.0000,0.000000,0.000000,8.0
6,36.156250,25.296875,0.939941,-1.0,-1.0,40.750000,20.437500,33.437500,20.218750,56.6875,...,25.234375,0.930176,18.953125,25.234375,0.000000,0.00000,0.0000,0.000000,0.000000,2.0
7,36.625000,29.656250,3.609375,1.0,-1.0,36.031250,29.859375,22.390625,31.890625,58.3750,...,34.468750,2.669922,0.000000,0.000000,17.046875,34.46875,0.0000,0.000000,0.000000,1.0
8,82.437500,43.156250,0.370117,-1.0,-1.0,92.750000,44.968750,88.812500,51.375000,101.9375,...,57.750000,1.540039,0.000000,0.000000,77.125000,57.75000,0.0000,0.000000,0.000000,0.0
9,63.187500,27.062500,1.669922,1.0,-1.0,63.625000,34.562500,56.343750,33.687500,72.4375,...,30.906250,3.449219,0.000000,0.000000,55.562500,40.15625,56.4375,21.625000,3.919922,5.0


## Data Set Construction for RF-Regression Model

In [4]:
    # file based constants
    features_dir     = '../data/general/feature-set/'
    features_regex   = re.compile(r'match\d+_features_2.csv')

In [5]:
    pool = multiprocessing.Pool()
    df = pd.DataFrame()

    
    features_csv_files = [f for f in listdir(features_dir) if features_regex.match(f)]
    shuffle(features_csv_files)
    
    print('All matches: {}'.format(len(features_csv_files)))
    features_all_csv = features_csv_files[0:len(features_csv_files)]
    df_all_data = pd.concat(pool.map(construct_data_set, features_all_csv))

    print('All-data-set shape: {}'.format(df_all_data.shape))
    
    describe_dataset(df_all_data)

All matches: 298
All-data-set shape: (679382, 230)
Shape: (679382, 230)

Pitch index counts
------------------
0	: 209737
1	: 19797
2	: 47201
3	: 18229
4	: 57806
5	: 169396
6	: 58463
7	: 21156
8	: 52785
9	: 24812


## Exporitng 

In [8]:
    df_all_data.to_csv('../data/general/{dataset}/data_{dataset}_3.csv'.format(dataset='all'), index=False)